# Sentiment analysis of tweets

Things to do:
- import all tweets as df (using existing methods)

In [1]:
import pandas as pd
import os
import sys
import time

In [2]:

parent_folder = os.path.abspath(os.path.join('..'))
csv_methods_folder = os.path.join(os.path.dirname(parent_folder), "tweet_csv_processing")
sys.path.append(csv_methods_folder)
print(csv_methods_folder)


/home/jabba/Desktop/Final-Project-Cryptocurrency-ML/tweet_csv_processing


In [3]:
import tweepy_csv_process as csv_process

In [4]:
csv_master_folder = os.path.join(os.path.dirname(parent_folder), "tweet-csv-cleaned")
test_day_folder = os.path.join(csv_master_folder, "2021-03-01")
test_file = os.path.join(test_day_folder, "2021-03-01-bitcoin-tweets.csv")

In [5]:
df = csv_process.dataframe_from_tweet_csv(test_file, "created_at")
print(df)

                                  id_str  \
created_at                                 
2021-03-01 00:00:00  1366176355713495041   
2021-03-01 00:00:00  1366176356732649480   
2021-03-01 00:00:01  1366176361694572544   
2021-03-01 00:00:01  1366176359664517121   
2021-03-01 00:00:01  1366176361757474816   
...                                  ...   
2021-03-01 23:59:51  1366538705750552579   
2021-03-01 23:59:52  1366538712507703299   
2021-03-01 23:59:52  1366538709529726977   
2021-03-01 23:59:52  1366538709961564163   
2021-03-01 23:59:55  1366538725182824454   

                                                from_user  \
created_at                                                  
2021-03-01 00:00:00                             Bitcoin 🤖   
2021-03-01 00:00:00                     Bitcoin Life Club   
2021-03-01 00:00:01    ₿itcoin ₿elly ₿utton ⚡️ 🔴🔴,🔴🔴🔴,🔴🔴🟠   
2021-03-01 00:00:01  Educación Financiera y Criptomonedas   
2021-03-01 00:00:01                             The First   


In [6]:
print(df.columns)

Index(['id_str', 'from_user', 'text', 'created_at', 'geo_coordinates',
       'user_lang', 'from_user_id_str', 'in_reply_to_status_id_str',
       'user_followers_count', 'user_location', 'hashtags'],
      dtype='object')


In [7]:
# So filter for 'en' tweets, remove links, hashtags, and non-english charecters
df_en_only = df.loc[df["user_lang"] == 'en']

In [8]:
df_en_only = df_en_only["text"]

In [9]:
df_en_only = df_en_only.drop_duplicates() 

In [63]:
import random
import re

df_len = len(df_en_only.index)
print(df_len)
random_index = random.randint(0, df_len)
print(f"tweet no. {random_index}")
random_tweet_text = df_en_only[random_index]
print(random_tweet_text)
print("-"*80)
# remove links

#remove mentions


def clean_text(text, remove_mentions=True, remove_hashtags=True, remove_links=True, filter_alphanumeric=True):
    cleaned = ""
    for word in text.split():
        if len(word) < 1:
            continue
        elif word[0] == '@' and remove_mentions:
            continue
        elif word[0] == '#' and remove_hashtags:
            continue
        elif word[0:8] == "https://" and remove_links:
            continue
        else:
            cleaned = cleaned +' '+ word
    if filter_alphanumeric:
        cleaned = re.sub(r'\W+', ' ', cleaned)
    return cleaned
    
cleaned = clean_text(random_tweet_text)
print(cleaned)
print("-"*80)

# this a regular expression which filters out non alpha-numeric chars
# this seems to filter out apostrophes too which is't always wanted... e.g. 'let's' -> 'let' 's'
# print(re.sub(r'\W+', ' ', cleaned))

30257
tweet no. 20006
#Bitcoin owners be like... #cryptoart #bitcoinnews https://t.co/pCJAukE2oa
--------------------------------------------------------------------------------
 owners be like 
--------------------------------------------------------------------------------


In [11]:
clean_df = df_en_only.apply(clean_text)

In [12]:
print(clean_df)

created_at
2021-03-01 00:00:00     Different stages of Money Evolution Read abov...
2021-03-01 00:00:01     The price of Spaghetti alla Carbonara is 24 2...
2021-03-01 00:00:01     You ve heard of amp but may not understand th...
2021-03-01 00:00:01     Thanks for this opportunity Explosive I like ...
2021-03-01 00:00:02     New Daily Candle BTC 45 103 00 3 32 ETH 1 416...
                                             ...                        
2021-03-01 23:59:48     When you take out the emotions the lines and ...
2021-03-01 23:59:51                                  My Bitcoin meme art
2021-03-01 23:59:52     141 1 in this optimal portfolio and 41 1 in t...
2021-03-01 23:59:52     Best risk adjusted returns in the past 24 hou...
2021-03-01 23:59:55                               braking out right now 
Name: text, Length: 30257, dtype: object


In [13]:
from textblob import TextBlob

In [76]:
df_len = len(clean_df.index)
print(df_len)
random_index = random.randint(0, df_len)
print(f"tweet no. {random_index}")
random_raw_text = df_en_only[random_index]
random_cleaned_text = clean_text(random_raw_text)
only_links_removed = clean_text(random_raw_text, remove_hashtags=False, remove_mentions=False)

print(random_raw_text)
print(TextBlob(random_raw_text).sentiment)
print(random_cleaned_text)
print(TextBlob(random_cleaned_text).sentiment)
print(TextBlob(only_links_removed).sentiment)


30258
tweet no. 4939
#linkedin #twitter #facebook #instagram #tiktok #socialmedia #pinterest #medium #deal #gift #gifts #giftideas #shop #deal #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/N4zGZzAgvt
Sentiment(polarity=0.0, subjectivity=0.0)
 on purchases over 25 at
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)


In [15]:
sentiment_df = clean_df
sentiment_df['sentiment'] = sentiment_df.apply(lambda text: TextBlob(text).sentiment)

In [16]:
# sentiment_df = sentiment_df.reset_index(drop=True)
sentiment_series = sentiment_df['sentiment'].tolist()

columns = ['polarity', 'subjectivity']

sentiment_only_df = pd.DataFrame(sentiment_series, columns=columns)

# print(sentiment_only_df)


In [17]:
print(sentiment_only_df)

       polarity  subjectivity
0      0.166667      0.400000
1      0.000000      0.000000
2     -0.343750      0.600000
3      0.100000      0.150000
4      0.068182      0.227273
...         ...           ...
30252  0.050000      0.581250
30253  0.000000      0.000000
30254 -0.125000      0.187500
30255  0.375000      0.275000
30256  0.285714      0.535714

[30257 rows x 2 columns]


In [18]:
polarities = sentiment_only_df['polarity'].tolist()
print(polarities)
print(sum(polarities)/len(polarities))

0000000000001, 0.4, 0.3181818181818182, 0.0, 0.0, 0.25, 0.75, -0.028333333333333332, 0.1, 0.25, 0.25, 0.11666666666666665, 0.0, 0.17045454545454544, 0.0, 0.0, 0.0, 0.0, 1.0, 0.65, 0.1, 1.0, 0.0, 0.5, 0.11666666666666665, 0.5571428571428572, 0.375, 0.8, 0.15000000000000002, 0.0, 0.16666666666666666, 0.4583333333333333, 0.0, 0.0, -0.3125, -0.13333333333333333, 0.38825757575757575, 0.29444444444444445, 0.0, -0.08333333333333336, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.075, 0.05, 0.0, -0.3125, 0.5333333333333333, 0.0, 0.0, 0.65, 0.0, 0.0, 0.0, -0.06666666666666667, 0.2, 0.0, 0.25, 1.0, 0.0, 0.5, 0.0, 0.0, 0.35, 0.2857142857142857, 0.7000000000000001, 0.0, 0.2222222222222222, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.75, 0.3, -0.125, 0.0, 0.0, 0.0, 0.08333333333333333, 0.0, 0.02053571428571429, 0.5, 0.10714285714285714, 0.15757575757575756, -0.025, 0.3333333333333333, -0.09999999999999998, 0.07169312169312168, 0.3071428571428572, 0.8, 0.0, 0.0, 5.551115123125783e-17, 0.0, 0.0, 0.037

In [19]:
def mean_polarity_subjectivity(sentiment_df, drop_zero_values=False):
    polarities = sentiment_df['polarity'].tolist()
    subjectivities = sentiment_df['subjectivity'].tolist()

    if drop_zero_values == True:
        polarities = list(filter(lambda x: x != 0, polarities))
        subjectivities = list(filter(lambda x: x != 0, subjectivities))

    mean_polarity = sum(polarities)/len(polarities)
    mean_subjectivity = sum(subjectivities)/len(subjectivities)

    return mean_polarity, mean_subjectivity

polarity, subjectivity = mean_polarity_subjectivity(sentiment_only_df)
polarity_no_zero, subjectivity_no_zero = mean_polarity_subjectivity(sentiment_only_df, drop_zero_values=True)

print(polarity, subjectivity)
print(polarity_no_zero, subjectivity_no_zero)


0.1154247993238734 0.3252909971562761
0.20435390012536203 0.5029294686232726


In [20]:
test_dict = {'date': ["23-23-23"], 'polarity': polarity, 'subjectivity': subjectivity}
test_df = pd.DataFrame(test_dict)
# print(test_df)

test_dict2 = {'date': ["13-23-23"], 'polarity': polarity_no_zero, 'subjectivity': subjectivity_no_zero}
test_df2 = pd.DataFrame(test_dict2)

test = pd.concat([test_df, test_df2], ignore_index=True)
print(test)

       date  polarity  subjectivity
0  23-23-23  0.115425      0.325291
1  13-23-23  0.204354      0.502929
